In [1]:
import gzip
import pandas as pd
import re

In [2]:
# download.geonames.org/export/zip/allCountries.zip

data_path='dataset/'
coords = pd.read_csv(data_path+'allCountries.txt', sep="\t", header=None)
coords = coords[[0,9,10]]
coords[9]  = [int(x*10)/10 for x in coords[9]]
coords[10] = [int(x*10)/10 for x in coords[10]]
coords=coords.applymap(str)
keys=coords[9]+'$'+coords[10]
values=coords[0]
check=dict(zip(keys,values))

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (1,4,6,7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
data_path='dataset/'
with open(data_path+'business.json', 'rb') as f:
    yelp = f.readlines()
yelp=''.join(str(yelp))
columns=["business_id", "name", "city", "state", "postal_code", "latitude", "longitude", "categories"]
data=pd.DataFrame(columns=columns)
for col in columns:
    data[col]=re.findall('"'+col+'": [\[]*(.*?["]*[\]]*["]*[^:]+),', yelp)
    data[col]=data[col].str.replace('"', '').str.replace("\\'s",'s').str.replace("\\","").str.replace('null', '-999')
    if col == 'latitude' or col == 'longitude':
        data[col]=data[col].astype(float)
data['intlat']  = [str(int(x*10)/10) for x in data['latitude']]
data['longlat'] = [str(int(x*10)/10) for x in data['longitude']]
data['key']=data['intlat']+'$'+data['longlat']
data[:3]

,business_id,name,city,state,postal_code,latitude,longitude,categories,intlat,longlat,key
0,FYWN1wneV18bWNgQjJ2GNg,Dental by Design,Ahwatukee,AZ,85044,33.330690,-111.978599,"Dentists, General Dentistry, Health & Medical,...",33.3,-111.9,33.3$-111.9
1,He-G7vWjzVUysIKrfNbPUQ,Stephen Szabo Salon,McMurray,PA,15317,40.291685,-80.104900,"Hair Stylists, Hair Salons, Mens Hair Salons, ...",40.2,-80.1,40.2$-80.1
2,KQPW8lFf1y5BT2MxiSZ3QA,Western Motor Vehicle,Phoenix,AZ,85017,33.524903,-112.115310,"Departments of Motor Vehicles, Public Services...",33.5,-112.1,33.5$-112.1


In [4]:
country=[]
for key in data['key']:
    try:
        country.append(check[key])
    except:
        country.append('')
data['country']=country
print('Fehlende: '+str(len(data[data['country']==''])))

Fehlende: 2941


In [5]:
miss=data[data['country']==''].copy()
data=data[data['country']!=''].copy()

In [6]:
# download.geonames.org/export/zip/allCountries.zip

data_path='dataset/'
codes = pd.read_csv(data_path+'allCountries.txt', sep="\t", header=None)
codes=codes[[0,1,2,3,4,5]].applymap(str)
codes.columns=['country','postal_code','district','state','state_short','county']

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (1,4,6,7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
# quick handle
codes['key']=codes['postal_code']+'$'+codes['state_short']+'$'+codes['district']
keys=codes['key'].tolist()
values=codes['country'].tolist()
check=dict(zip(keys,values))

codes['short']=codes['postal_code']+'$'+codes['state_short']
keys=codes['short'].tolist()
short=dict(zip(keys,values))

codes['short2']=codes['postal_code']+'$'+codes['district']
keys=codes['short2'].tolist()
short2=dict(zip(keys,values))

def check_zip(postal, state, city):
    postal=str(postal).strip().split(' ')[0]  
    
    #quick
    try:
        return check[postal+'$'+state+'$'+city]
    
    #quick
    except:
        try:
            return short[postal+'$'+state]
        #quick
        except:
            try:
                return short2[postal+'$'+city]
            except:  
                return ''
            
            #super-slow
            """
            match=codes[codes['postal_code']==postal]
            match=match[match['state_short']==state]
            match=match[match['district'].str.contains(city)]
            match=list(set(match['country'].tolist()))
            l=len(match)
            if l==0:
                return 'not found'
            elif l==1:
                return match[0]
            else:
                return 'multiple results'
            """

In [8]:
#example
check_zip(85044,'AZ', 'Phoenix'),check_zip('M4M 3A6','ON','Toronto'),check_zip(70567,'BW','Stuttgart')

('US', 'CA', 'DE')

In [9]:
country=[]
for index, row in miss.iterrows():
    country.append(check_zip(row['postal_code'], row['state'], row['city']))

In [10]:
miss['country']=country
data=data.append(miss)

data['intlat2']  = [str(int(x)) for x in data['latitude']]
data['longlat2'] = [str(int(x)) for x in data['longitude']]

data['coord2']=data['intlat2']+'$'+data['longlat2']


miss=data[data['country']==''].copy()
data=data[data['country']!=''].copy()

keys=data['coord2']
values=data['country']
check=dict(zip(keys,values))

print('Fehlende: '+str(len(miss)))

Fehlende: 27


In [11]:
country=[]
for c in miss['coord2']:
    try:
        country.append(check[c])
    except:
        country.append('')

In [12]:
miss['country']=country
data=data.append(miss)

data['intlat2']  = [str(int(x)) for x in data['latitude']]
data['longlat2'] = [str(int(x)) for x in data['longitude']]

data['coord2']=data['intlat2']+'$'+data['longlat2']


miss=data[data['country']==''].copy()
data=data[data['country']!=''].copy()

keys=data['coord2']
values=data['country']
check=dict(zip(keys,values))

print('Fehlende: '+str(len(miss)))

Fehlende: 10


In [13]:
miss[columns+['country']]

,business_id,name,city,state,postal_code,latitude,longitude,categories,country
17025,W2iWAyXd48iRkCKWOmZZuw,Viu00f1amar,Casablanca,VS,2480000,-33.353150,-71.357290,"Food, Wineries, Arts & Entertainment]",
19121,op_1wVF_vxbjnp_gBW9d1Q,Bodegas RE,Casablanca,VS,2340000,-33.226921,-71.467934,"Arts & Entertainment, Wineries, Food]",
44939,VcM3_Va-SfkM35XTAV1Idw,Emiliana,Casablanca,VS,,-33.401111,-71.115401,"Food, Wineries, Beer, Wine & Spirits, Arts & E...",
90421,1yQUqh3_h1IOrXZmb4CBFw,TriBeCa,Edinburgh,EDH,EH10 4HG,89.999314,-142.466650,"Restaurants, Breakfast & Brunch, American (Tra...",
123335,YSEW66iMAheOH1x6YYZEPA,Viu00f1a Veramonte,Casablanca,VS,,-33.370547,-71.294938,"Food, Cafes, Restaurants, Wine Tasting Room, C...",
124753,MImmd7Y7K3kCAP0YG05hOQ,Macerado Restaurante,Casablanca,VS,2480000,-33.327618,-71.394905,Restaurants],
142468,yyF9HvCBuaTa8fMnpN-2-A,Viu00f1a Indu00f3mita,Casablanca,VS,,-33.316431,-71.400177,],
154225,vBxK_MAGuy8eWL_CCfUCUQ,Lone Star,George Town,CHE,CW12 4EU,19.327681,-81.381087,"Nightlife, Bars, Restaurants, Tex-Mex, Sports ...",
159672,6S3sGf8PC5dymj7VI37cuA,Al Settimo Arte - La Carpe,Saint-sauveur,WHT,7912,50.708270,3.598120,"Pizza, Italian, Restaurants]",
174465,rQY4dgr_juFUm4znznGmLQ,Tanino,Casablanca,VS,,-33.317491,-71.434989,"Restaurants, Argentine, Signature Cuisine]",


In [14]:
data=data[columns+['country']]
data[:3] 

,business_id,name,city,state,postal_code,latitude,longitude,categories,country
0,FYWN1wneV18bWNgQjJ2GNg,Dental by Design,Ahwatukee,AZ,85044,33.330690,-111.978599,"Dentists, General Dentistry, Health & Medical,...",US
1,He-G7vWjzVUysIKrfNbPUQ,Stephen Szabo Salon,McMurray,PA,15317,40.291685,-80.104900,"Hair Stylists, Hair Salons, Mens Hair Salons, ...",US
2,KQPW8lFf1y5BT2MxiSZ3QA,Western Motor Vehicle,Phoenix,AZ,85017,33.524903,-112.115310,"Departments of Motor Vehicles, Public Services...",US


In [15]:
data['country'].value_counts()

US    130332
CA     38378
GB      4684
DE      1111
AR        29
NO        10
AT         4
IE         3
ES         2
MX         1
FR         1
IT         1
MD         1
Name: country, dtype: int64